In [67]:
import os
import pickle
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.functional as F
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from meta_load import *
from copy import copy

In [2]:
df = load_meta_v2('C:\\Users\\abdig\\Desktop\\NIRLBCM')
df

5 4133
(4133, 30) (4133,) (4133,) (4133,)


,Senteracorrelation15_0,Senteracorrelation15_90,Senteracorrelation30_0,Senteracorrelation30_90,Senteracorrelation50_0,Senteracorrelation50_90,Senteradissimilarity15_0,Senteradissimilarity15_90,Senteradissimilarity30_0,Senteradissimilarity30_90,...,Senteraenergy50_90,Senterahomogeneity15_0,Senterahomogeneity15_90,Senterahomogeneity30_0,Senterahomogeneity30_90,Senterahomogeneity50_0,Senterahomogeneity50_90,fly,landmark,Place
0,0.476477,0.465833,0.409301,0.367359,0.384854,0.346171,5.170855,5.315930,5.944262,6.328646,...,0.039397,0.212950,0.207522,0.182443,0.172940,0.173041,0.165915,8,N_Control,10_A
1,0.596765,0.603557,0.535524,0.513136,0.510215,0.469953,3.939831,3.869886,4.540472,4.672118,...,0.049658,0.258342,0.260609,0.227956,0.224089,0.218585,0.210886,8,P_Control,10_B
2,0.902374,0.808721,0.894170,0.685240,0.889307,0.507987,3.392961,4.017774,3.845818,5.322311,...,0.058415,0.305462,0.298824,0.272320,0.252142,0.261560,0.234394,8,K_Control,10_C
3,0.455300,0.458746,0.407964,0.332222,0.379293,0.327955,4.319543,4.135542,4.713979,4.983386,...,0.051854,0.238563,0.248382,0.215524,0.210438,0.206420,0.204328,8,Control,10_D
4,0.359419,0.358603,0.264664,0.205188,0.232540,0.172992,4.447881,4.458017,5.226406,5.580744,...,0.050134,0.235793,0.234792,0.199727,0.191236,0.190263,0.181500,8,H50%,10_E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,0.107939,0.076203,0.070492,0.034105,0.055726,0.048022,7.435428,7.886051,7.922922,8.348259,...,0.048814,0.192808,0.184500,0.180011,0.173541,0.175187,0.173564,12,K_Control,9_C
4129,0.096622,0.092630,0.079991,0.066117,0.073162,0.065683,8.367630,8.475564,8.764049,9.029901,...,0.041599,0.178869,0.176600,0.165965,0.162432,0.160716,0.160879,12,Control,9_D
4130,0.067278,0.058605,0.029753,0.010366,0.026378,0.012743,8.416477,8.675199,9.025114,9.517810,...,0.042555,0.174063,0.168646,0.158864,0.152074,0.155924,0.149489,12,H50%,9_E
4131,0.504044,0.365512,0.493345,0.227730,0.485458,0.097786,7.149043,8.083791,7.426701,8.922921,...,0.052214,0.210077,0.200988,0.200621,0.186061,0.196962,0.178991,12,H75%,9_F


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [15]:
N_data = df[(df['landmark']=='N_Control') | (df['landmark']=='N_Exceso') | (df['landmark']=='N_Deficiencia')]
#optional
#N_data = N_data.iloc[:,[4,5,10,11,16,17,22,23,28,29,30,31,32]]
print(N_data.columns)
N_data = N_data[N_data['fly']!=0]
N_data

Index(['Senteracorrelation15_0', 'Senteracorrelation15_90',
       'Senteracorrelation30_0', 'Senteracorrelation30_90',
       'Senteracorrelation50_0', 'Senteracorrelation50_90',
       'Senteradissimilarity15_0', 'Senteradissimilarity15_90',
       'Senteradissimilarity30_0', 'Senteradissimilarity30_90',
       'Senteradissimilarity50_0', 'Senteradissimilarity50_90',
       'Senteracontrast15_0', 'Senteracontrast15_90', 'Senteracontrast30_0',
       'Senteracontrast30_90', 'Senteracontrast50_0', 'Senteracontrast50_90',
       'Senteraenergy15_0', 'Senteraenergy15_90', 'Senteraenergy30_0',
       'Senteraenergy30_90', 'Senteraenergy50_0', 'Senteraenergy50_90',
       'Senterahomogeneity15_0', 'Senterahomogeneity15_90',
       'Senterahomogeneity30_0', 'Senterahomogeneity30_90',
       'Senterahomogeneity50_0', 'Senterahomogeneity50_90', 'fly', 'landmark',
       'Place'],
      dtype='object')


,Senteracorrelation15_0,Senteracorrelation15_90,Senteracorrelation30_0,Senteracorrelation30_90,Senteracorrelation50_0,Senteracorrelation50_90,Senteradissimilarity15_0,Senteradissimilarity15_90,Senteradissimilarity30_0,Senteradissimilarity30_90,...,Senteraenergy50_90,Senterahomogeneity15_0,Senterahomogeneity15_90,Senterahomogeneity30_0,Senterahomogeneity30_90,Senterahomogeneity50_0,Senterahomogeneity50_90,fly,landmark,Place
0,0.476477,0.465833,0.409301,0.367359,0.384854,0.346171,5.170855,5.315930,5.944262,6.328646,...,0.039397,0.212950,0.207522,0.182443,0.172940,0.173041,0.165915,8,N_Control,10_A
5,0.717791,0.836725,0.593990,0.810286,0.526856,0.797638,5.224901,4.484461,6.145069,5.316307,...,0.041786,0.225996,0.238515,0.197559,0.205781,0.188111,0.196937,8,N_Control,11_A
14,0.961340,0.899403,0.952348,0.820458,0.947854,0.712307,3.701282,4.343193,4.338931,5.738532,...,0.056901,0.291313,0.285555,0.261329,0.247705,0.252379,0.227727,8,N_Control,13_A
19,0.411516,0.432768,0.364416,0.354527,0.344122,0.328885,4.945966,4.655122,5.563750,5.699782,...,0.048131,0.229910,0.236639,0.199898,0.195701,0.192642,0.188504,8,N_Control,14_A
25,0.958858,0.842244,0.952019,0.688759,0.949215,0.427593,3.266779,3.808897,3.637156,5.070382,...,0.063396,0.301918,0.303465,0.279693,0.265762,0.270912,0.246369,8,N_Deficiencia,15_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4098,0.880090,0.852024,0.874244,0.817857,0.872741,0.780987,7.013235,7.941183,7.556414,9.365093,...,0.127696,0.298556,0.291875,0.286607,0.270819,0.282418,0.258503,12,N_Exceso,5_A
4105,0.531640,0.515244,0.496358,0.449487,0.488698,0.389998,13.019881,13.521985,14.314616,15.344018,...,0.046334,0.166294,0.161350,0.149463,0.140921,0.145904,0.131400,12,N_Exceso,6_A
4112,0.138381,0.097962,0.096808,0.061318,0.084060,0.083030,6.112596,6.399210,6.512383,6.656830,...,0.064825,0.224303,0.223240,0.214393,0.215024,0.209619,0.212708,12,N_Exceso,7_A
4119,0.150908,0.135199,0.095446,0.063742,0.072218,0.044371,7.769268,8.038002,8.570655,9.056187,...,0.039000,0.178034,0.174124,0.159255,0.152799,0.151822,0.148008,12,N_Control,8_A


In [16]:
n2clas={'N_Deficiencia':0, 'N_Control':1, 'N_Exceso':2}

In [17]:
N_data.landmark.replace(n2clas, inplace=True)
N_data

,Senteracorrelation15_0,Senteracorrelation15_90,Senteracorrelation30_0,Senteracorrelation30_90,Senteracorrelation50_0,Senteracorrelation50_90,Senteradissimilarity15_0,Senteradissimilarity15_90,Senteradissimilarity30_0,Senteradissimilarity30_90,...,Senteraenergy50_90,Senterahomogeneity15_0,Senterahomogeneity15_90,Senterahomogeneity30_0,Senterahomogeneity30_90,Senterahomogeneity50_0,Senterahomogeneity50_90,fly,landmark,Place
0,0.476477,0.465833,0.409301,0.367359,0.384854,0.346171,5.170855,5.315930,5.944262,6.328646,...,0.039397,0.212950,0.207522,0.182443,0.172940,0.173041,0.165915,8,1,10_A
5,0.717791,0.836725,0.593990,0.810286,0.526856,0.797638,5.224901,4.484461,6.145069,5.316307,...,0.041786,0.225996,0.238515,0.197559,0.205781,0.188111,0.196937,8,1,11_A
14,0.961340,0.899403,0.952348,0.820458,0.947854,0.712307,3.701282,4.343193,4.338931,5.738532,...,0.056901,0.291313,0.285555,0.261329,0.247705,0.252379,0.227727,8,1,13_A
19,0.411516,0.432768,0.364416,0.354527,0.344122,0.328885,4.945966,4.655122,5.563750,5.699782,...,0.048131,0.229910,0.236639,0.199898,0.195701,0.192642,0.188504,8,1,14_A
25,0.958858,0.842244,0.952019,0.688759,0.949215,0.427593,3.266779,3.808897,3.637156,5.070382,...,0.063396,0.301918,0.303465,0.279693,0.265762,0.270912,0.246369,8,0,15_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4098,0.880090,0.852024,0.874244,0.817857,0.872741,0.780987,7.013235,7.941183,7.556414,9.365093,...,0.127696,0.298556,0.291875,0.286607,0.270819,0.282418,0.258503,12,2,5_A
4105,0.531640,0.515244,0.496358,0.449487,0.488698,0.389998,13.019881,13.521985,14.314616,15.344018,...,0.046334,0.166294,0.161350,0.149463,0.140921,0.145904,0.131400,12,2,6_A
4112,0.138381,0.097962,0.096808,0.061318,0.084060,0.083030,6.112596,6.399210,6.512383,6.656830,...,0.064825,0.224303,0.223240,0.214393,0.215024,0.209619,0.212708,12,2,7_A
4119,0.150908,0.135199,0.095446,0.063742,0.072218,0.044371,7.769268,8.038002,8.570655,9.056187,...,0.039000,0.178034,0.174124,0.159255,0.152799,0.151822,0.148008,12,1,8_A


In [18]:
N_data.shape

(540, 33)

In [19]:

class datapaltas(Dataset):
    def __init__(self, df=N_data, scale =True):
        self.df = df

    def __getitem__(self, index):
        X = (self.df.iloc[index,:-2]).values
        X = X.astype(np.float64)
        X = torch.from_numpy(X).float()
        y = self.df.iloc[index,-2]
        #print(y)
        y = torch.Tensor([y]).long()
        return X, y
    
    def __len__(self):
        return self.df.shape[0]

In [31]:
class SimpleClassification(nn.Module):
    
    def __init__(self, n_features = 11, n_classes = 3):
        super(SimpleClassification, self).__init__()
        self.n_features = n_features
        self.l1 = nn.Linear(self.n_features, 128)
        self.l2 = nn.Linear(128, 64)
        self.l3 = nn.Linear(64, 32)
        self.l4 = nn.Linear(32, 16)
        self.l5 = nn.Linear(16, n_classes)
        
        self.relu = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm1d(128)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.batchnorm3 = nn.BatchNorm1d(32)
        self.batchnorm4 = nn.BatchNorm1d(16)
    
    def forward(self, x):
        x = self.relu(self.batchnorm1(self.l1(x)))
        x = self.relu(self.batchnorm2(self.l2(x)))
        x = self.relu(self.batchnorm3(self.l3(x)))
        x = self.relu(self.batchnorm4(self.l4(x)))
        return self.l5(x)

In [32]:
a = datapaltas()
a[0], a[4]

((tensor([4.7648e-01, 4.6583e-01, 4.0930e-01, 3.6736e-01, 3.8485e-01, 3.4617e-01,
          5.1709e+00, 5.3159e+00, 5.9443e+00, 6.3286e+00, 6.2214e+00, 6.5369e+00,
          1.0851e+02, 1.1067e+02, 1.2234e+02, 1.3108e+02, 1.2760e+02, 1.3548e+02,
          4.3466e-02, 4.3062e-02, 4.0800e-02, 4.0031e-02, 3.9984e-02, 3.9397e-02,
          2.1295e-01, 2.0752e-01, 1.8244e-01, 1.7294e-01, 1.7304e-01, 1.6592e-01,
          8.0000e+00]),
  tensor([1])),
 (tensor([9.5886e-01, 8.4224e-01, 9.5202e-01, 6.8876e-01, 9.4921e-01, 4.2759e-01,
          3.2668e+00, 3.8089e+00, 3.6372e+00, 5.0704e+00, 3.7932e+00, 6.2706e+00,
          2.7833e+01, 9.9689e+01, 3.2480e+01, 1.8232e+02, 3.4403e+01, 2.8781e+02,
          6.9708e-02, 6.8522e-02, 6.7845e-02, 6.4792e-02, 6.6879e-02, 6.3396e-02,
          3.0192e-01, 3.0347e-01, 2.7969e-01, 2.6576e-01, 2.7091e-01, 2.4637e-01,
          8.0000e+00]),
  tensor([0])))

In [37]:
a = datapaltas()
train_loader = DataLoader(dataset=a, batch_size=64)
model = SimpleClassification(n_features=31)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
print(model)


SimpleClassification(
  (l1): Linear(in_features=31, out_features=128, bias=True)
  (l2): Linear(in_features=128, out_features=64, bias=True)
  (l3): Linear(in_features=64, out_features=32, bias=True)
  (l4): Linear(in_features=32, out_features=16, bias=True)
  (l5): Linear(in_features=16, out_features=3, bias=True)
  (relu): ReLU()
  (batchnorm1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [90]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc

def F1_score(prob, label):
    y_pred_softmax = torch.log_softmax(prob, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
    #print(y_pred_tags, label)
    return f1_score(label.cpu(), y_pred_tags.cpu(), average='macro')

In [121]:
def train(idx, ep = 100):
    idx_result = {'idx_used':copy(idx), 'f1': 0, 'acc':0}
    idx_copy = copy(idx)
    aux_f1_idx = []
    aux_ac_idx = []
    n_f = len(idx_copy)
    idx_copy.extend([-2, -1])
    a = datapaltas(df = N_data.iloc[:, idx_copy])
    EPOCHS = ep
    
    skf = StratifiedKFold()
    skf.get_n_splits(N_data, N_data.iloc[:,-2])
    for fold, (train_ids, test_ids) in enumerate(skf.split(N_data, N_data.iloc[:,-2])):
        l_v = []
        a_v = []
        f_v = []
        #print(f'Fold {fold+1}')
        #print('------------------------')
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
        train_loader = torch.utils.data.DataLoader(
                          a, 
                          batch_size=64, sampler=train_subsampler)
        test_loader = torch.utils.data.DataLoader(
                          a,
                          batch_size=1, sampler=test_subsampler)
        model = SimpleClassification(n_features=n_f)
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr = 0.001)
        itt = tqdm(range(EPOCHS))
        for i in itt:
            loss_epoch = 0
            acc_train, acc_test = 0.0, 0.0
            f1_train, f1_test = 0.0, 0.0
            model.train()
            for X, y in train_loader:
                X, y = X.to(device), y.to(device).flatten()
                optimizer.zero_grad()
                y_pred = model(X)
                loss = criterion(y_pred, y)
                acc = multi_acc(y_pred, y)
                f1 = F1_score(y_pred, y)
                loss.backward()
                optimizer.step()
                loss_epoch += loss.item()
                acc_train += acc.item()
                f1_train += f1.item()

            with torch.no_grad():

                for X, y in test_loader:
                    X, y = X.to(device), y.to(device).flatten()
                    model.eval()
                    output = model(X)
                    acc = multi_acc(output, y)
                    f1 = F1_score(output, y)
                    acc_test += acc.item()
                    f1_test += f1.item()

            itt.set_description("Acc train: %.2f Acc test: %.2f F1 train: %.2f F1 test: %.2f" % (acc_train/len(train_loader), acc_test/len(test_loader), f1_train/len(train_loader), f1_test/len(test_loader)))

            l_v.append(loss_epoch)
            a_v.append(acc_test/len(test_loader))
            f_v.append(f1_test/len(test_loader))
            
            #print(f'Loss {i+1}: {loss_epoch}')
            #print(f'Acc {i+1}: {acc_epoch/len(train_loader)}')
            #print(f'F1 {i+1}: {f1_epoch/len(train_loader)}')
        #print(a_v, f_v)
        aux_f1_idx.append(f_v)
        aux_ac_idx.append(a_v)
        
    aux_ac_idx = np.array(aux_ac_idx)
    aux_f1_idx = np.array(aux_f1_idx)
    #print(aux_ac_idx, aux_f1_idx)
    idx_result['f1'] = max(np.mean(aux_f1_idx, axis = 0))
    idx_result['acc'] = max(np.mean(aux_ac_idx, axis = 0))
    return idx_result
        #plt.figure(), plt.plot(l_v), plt.title('Loss by epoch')
        #plt.figure(), plt.plot(a_v), plt.title('Accuracy test by epoch')
        #plt.figure(), plt.plot(f_v), plt.title('F1 test by epoch')
        #plt.show()

In [122]:
train([0], ep = 3)

Acc train: 40.00 Acc test: 34.26 F1 train: 0.32 F1 test: 0.34: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.16it/s]
Acc train: 40.29 Acc test: 36.11 F1 train: 0.34 F1 test: 0.36: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.12it/s]
Acc train: 38.29 Acc test: 35.19 F1 train: 0.30 F1 test: 0.35: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.15it/s]
Acc train: 32.57 Acc test: 34.26 F1 train: 0.31 F1 test: 0.34: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.06it/s]
Acc train: 39.57 Acc test: 35.19 F1 train: 0.35 F1 test: 0.35: 100%|████████████████████████████████████████████████████████████████████████████████

{'idx_used': [0], 'f1': 0.35, 'acc': 35.00000000000001}

In [119]:
N_FEATURES = 31
def select_k_best(k = 10, ev = 'f1'):
    """
    ev = 'acc' or 'f1'
    """
    total_log = []
    list_features = list(range(N_FEATURES))
    initial_feat = []
    while(len(initial_feat)<k):
        selector = {}
        for i in list_features:
            initial_feat.append(i)
            print(f"=======================================Using {initial_feat} ===============")
            res = train(initial_feat, ep = 100)
            initial_feat.pop()
            selector.update({i:res[ev]})
            total_log.append(res)
        best_feature = max(selector, key=selector.get)
        initial_feat.append(best_feature)
        list_features.remove(best_feature)
        print("================================================")
        print(f"=================ADEDD {max(selector, key=selector.get)} TO BEST FEATURES ==========")
        print(f"================={ev} = {selector[best_feature]} ================================")
        
    print(f"FINAL FEATURES")
    print(initial_feat)

In [120]:
select_k_best(k = 10)

=======================================Using [0] ===============


Acc train: 32.43 Acc test: 34.26 F1 train: 0.29 F1 test: 0.34: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.05it/s]
Acc train: 42.29 Acc test: 33.33 F1 train: 0.35 F1 test: 0.33: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.01it/s]
Acc train: 36.71 Acc test: 35.19 F1 train: 0.28 F1 test: 0.35: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.01it/s]
Acc train: 38.00 Acc test: 35.19 F1 train: 0.31 F1 test: 0.35: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.11it/s]
Acc train: 36.29 Acc test: 38.89 F1 train: 0.32 F1 test: 0.39: 100%|████████████████████████████████████████████████████████████████████████████████

=======================================Using [1] ===============


Acc train: 38.43 Acc test: 34.26 F1 train: 0.29 F1 test: 0.34: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.04it/s]
Acc train: 39.86 Acc test: 34.26 F1 train: 0.32 F1 test: 0.34: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.01it/s]
Acc train: 39.29 Acc test: 35.19 F1 train: 0.31 F1 test: 0.35: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.66it/s]
Acc train: 31.71 Acc test: 30.56 F1 train: 0.25 F1 test: 0.31: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.92it/s]
Acc train: 38.86 Acc test: 29.63 F1 train: 0.38 F1 test: 0.30: 100%|████████████████████████████████████████████████████████████████████████████████

=======================================Using [2] ===============


...

=======================================Using [0, 1, 3] ===============


Acc train: 45.71 Acc test: 34.26 F1 train: 0.45 F1 test: 0.34: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.29it/s]
Acc train: 45.43 Acc test: 31.48 F1 train: 0.42 F1 test: 0.31: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.59it/s]
Acc train: 49.57 Acc test: 34.26 F1 train: 0.42 F1 test: 0.34: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.16it/s]
Acc train: 44.00 Acc test: 36.11 F1 train: 0.42 F1 test: 0.36: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.11it/s]
Acc train: 49.57 Acc test: 37.96 F1 train: 0.45 F1 test: 0.38: 100%|████████████████████████████████████████████████████████████████████████████████

=======================================Using [0, 1, 4] ===============


Acc train: 47.86 Acc test: 36.11 F1 train: 0.45 F1 test: 0.36: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.56it/s]
Acc train: 47.57 Acc test: 37.04 F1 train: 0.43 F1 test: 0.37: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.14it/s]
Acc train: 35.29 Acc test: 35.19 F1 train: 0.19 F1 test: 0.35: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.08it/s]
Acc train: 43.71 Acc test: 35.19 F1 train: 0.43 F1 test: 0.35: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.27it/s]
Acc train: 42.86 Acc test: 36.11 F1 train: 0.37 F1 test: 0.36: 100%|████████████████████████████████████████████████████████████████████████████████

=================ADEDD 4 TO BEST FEATURES ==========
=================f1 = 0.3592592592592593 ================================
